In [1]:
import pandas as pd
import numpy as np

### RBD example 1: 

A large catalogue chain store has been experimenting with several methods of advertising its extensive variety of bicycles. 

Three kinds of catalogues have been prepared: (1) a side view of each bicycle is shown, (2 each bicycle’s excellent record of longevity is extolled, (3)) pictures of the bicycles with their riders are shown. 

The company’s management would like to know if there are differences in sales among the stores that use the different catalogues. The monthly sales of bicycles of three randomly selected stores, each using a different catalogue, are shown in *RBD_ex1.xlsx*. Do these data allow us to conclude that there are differences in bicycle sales among the stores using the three catalogues?

In [2]:
# Step 1: Load data
file_path = "data/RBD_ex1.xlsx"  # update path if needed
df = pd.read_excel(file_path, skiprows=2)
df = df.loc[:, df.columns[:4]]  # keep only the first 4 columns
df.columns = ['Month', 'Catalogue_1', 'Catalogue_2', 'Catalogue_3']
df['Month'] = pd.to_datetime(df['Month'])

# Step 2: Extract numerical data (rows = months, columns = catalogues)
data = df[['Catalogue_1', 'Catalogue_2', 'Catalogue_3']].to_numpy()
a = data.shape[1]  # number of treatments (catalogues)
b = data.shape[0]  # number of blocks (months)
N = a * b

# Step 3: Means
grand_mean = data.mean()
treatment_means = data.mean(axis=0)  # mean per column (catalogue)
block_means = data.mean(axis=1)      # mean per row (month)

# Step 4: Sum of Squares
SS_treatment = b * np.sum((treatment_means - grand_mean) ** 2)
SS_block = a * np.sum((block_means - grand_mean) ** 2)
SS_total = np.sum((data - grand_mean) ** 2)
SS_error = SS_total - SS_treatment - SS_block

# Step 5: Degrees of Freedom
df_treatment = a - 1
df_block = b - 1
df_error = df_treatment * df_block

# Step 6: Mean Squares
MS_treatment = SS_treatment / df_treatment
MS_block = SS_block / df_block
MS_error = SS_error / df_error

# Step 7: F-statistics
F_treatment = MS_treatment / MS_error
F_block = MS_block / MS_error

# Step 8: Construct ANOVA Table
anova_table = pd.DataFrame({
    'Source': ['Treatment (Catalogue)', 'Block (Month)', 'Error'],
    'SS': [SS_treatment, SS_block, SS_error],
    'df': [df_treatment, df_block, df_error],
    'MS': [MS_treatment, MS_block, MS_error],
    'F': [F_treatment, F_block, np.nan]
})

print(anova_table)

                  Source             SS  df            MS          F
0  Treatment (Catalogue)     523.083333   2    261.541667   0.437975
1          Block (Month)  205134.625000   7  29304.946429  49.073802
2                  Error    8360.250000  14    597.160714        NaN


### RBD example 2:

A tire company conducted tests on tire surfaces to determine whether there is a significant difference in rubber damage when the average speed of the automobile varies.
* Independent variable: speed of automobile
* 3reatment levels: slow speed (20 miles/h), medium speed (40 miles/h), and high speed (60 miles/h)
* The company uses 5 suppliers of the rubber from which the tires are made.
* To control for this experimentally, the analysts used supplier as a blocking variable.
* 15 tires were randomly selected for the study, 3 from each supplier.
* Each of the 3 was assigned to be tested under a different speed condition.

In [3]:
# Step 1: Load data in original format (treatments = rows, blocks = columns)
file_path = "data/RBD_ex2.xlsx"
df = pd.read_excel(file_path, skiprows=3, header=None)
df.columns = ['Speed', 'S1', 'S2', 'S3', 'S4', 'S5']

# Step 2: Extract numerical data (exclude 'Speed' column)
data = df[['S1', 'S2', 'S3', 'S4', 'S5']].to_numpy()
a = data.shape[0]  # number of treatments (speeds = rows)
b = data.shape[1]  # number of blocks (suppliers = columns)
N = a * b

# Step 3: Means
grand_mean = data.mean()
treatment_means = data.mean(axis=1)  # mean of each row (speed)
block_means = data.mean(axis=0)      # mean of each column (supplier)

# Step 4: Sum of Squares
SS_treatment = b * np.sum((treatment_means - grand_mean)**2)
SS_block = a * np.sum((block_means - grand_mean)**2)
SS_total = np.sum((data - grand_mean)**2)
SS_error = SS_total - SS_treatment - SS_block

# Step 5: Degrees of Freedom
df_treatment = a - 1
df_block = b - 1
df_error = (a - 1) * (b - 1)

# Step 6: Mean Squares
MS_treatment = SS_treatment / df_treatment
MS_block = SS_block / df_block
MS_error = SS_error / df_error

# Step 7: F-Statistics
F_treatment = MS_treatment / MS_error
F_block = MS_block / MS_error

# Step 8: Assemble ANOVA Table
anova_table = pd.DataFrame({
    'Source': ['Treatment (Speed)', 'Block (Supplier)', 'Error'],
    'SS': [SS_treatment, SS_block, SS_error],
    'df': [df_treatment, df_block, df_error],
    'MS': [MS_treatment, MS_block, MS_error],
    'F': [F_treatment, F_block, np.nan]
})

print(anova_table)


              Source        SS  df        MS          F
0  Treatment (Speed)  3.484000   2  1.742000  97.682243
1   Block (Supplier)  1.549333   4  0.387333  21.719626
2              Error  0.142667   8  0.017833        NaN


# Two-way ANOVA example:

Problem 2 in Assignment 3

Employees of a corporation are about to undergo a retraining program. Management is trying to determine which of three programs is the best. They believe that the effectiveness of the programs may be influenced by the age group. A factorial experiment was designed.

In [4]:
# Step 1: Load Excel file
file_path = "data/Factorial_Retraining_Program.xlsx"
df = pd.read_excel(file_path)

# Step 2: Prepare a data matrix with shape (programs × age groups × replicates)
programs = df['Program'].unique()
age_groups = df['Age Group'].unique()
replicates = 2

# 3D array: programs × age_groups × replicates
data_tensor = np.empty((len(programs), len(age_groups), replicates))

# Fill the tensor
for i, prog in enumerate(programs):
    for j, age in enumerate(age_groups):
        scores = df[(df['Program'] == prog) & (df['Age Group'] == age)]['Score'].values
        data_tensor[i, j, :] = scores

# Step 3: Compute means
grand_mean = data_tensor.mean()
a_means = data_tensor.mean(axis=(1,2))  # mean for each program
b_means = data_tensor.mean(axis=(0,2))  # mean for each age group
cell_means = data_tensor.mean(axis=2)   # mean for each (program, age group) cell

# Step 4: Sum of squares
a = len(programs)
b = len(age_groups)
n = replicates

# SS for Program (A)
SSA = b * n * np.sum((a_means - grand_mean)**2)

# SS for Age Group (B)
SSB = a * n * np.sum((b_means - grand_mean)**2)

# SS for Interaction (AB)
SSAB = n * np.sum((cell_means - a_means[:, None] - b_means[None, :] + grand_mean)**2)

# SS Total
SST = np.sum((data_tensor - grand_mean)**2)

# SS Error
SSE = SST - SSA - SSB - SSAB

# Step 5: Degrees of freedom
df_A = a - 1
df_B = b - 1
df_AB = df_A * df_B
df_E = a * b * (n - 1)

# Step 6: Mean Squares
MSA = SSA / df_A
MSB = SSB / df_B
MSAB = SSAB / df_AB
MSE = SSE / df_E

# Step 7: F-statistics
F_A = MSA / MSE
F_B = MSB / MSE
F_AB = MSAB / MSE

# Step 8: ANOVA Table
anova_table = pd.DataFrame({
    'Source': ['Program', 'Age Group', 'Interaction', 'Error'],
    'SS': [SSA, SSB, SSAB, SSE],
    'df': [df_A, df_B, df_AB, df_E],
    'MS': [MSA, MSB, MSAB, MSE],
    'F': [F_A, F_B, F_AB, np.nan]
})

print(anova_table)


        Source            SS  df            MS          F
0      Program  36150.000000   2  18075.000000  12.758824
1    Age Group  16133.333333   1  16133.333333  11.388235
2  Interaction   1516.666667   2    758.333333   0.535294
3        Error   8500.000000   6   1416.666667        NaN
